In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
from pathlib import Path

# ==================== 1. CONFIGURACIÓN ====================
INPUT_FILE = '../data/processed/f1_features_complete.csv'
OUTPUT_DIR = '../models'
FINAL_MODEL_FILE = f'{OUTPUT_DIR}/xgboost_base_final.pkl' 

# TUS PARÁMETROS GANADORES (Copiados de tu resultado 4.4)
best_params = {
    'subsample': 0.95,
    'reg_alpha': 0.1,
    'n_estimators': 50,
    'max_depth': 3,
    'learning_rate': 0.08,
    'colsample_bytree': 0.5,
    'objective': 'reg:squarederror',
    'n_jobs': -1,
    'random_state': 42
}

# Las features exactas que usamos
feature_cols = [
    'pct_puntos_actual', 'pct_linear_points', 'posicion_media',
    'tendencia_ultimas_3', 'diff_con_lider_normalizada', 'progreso_temporada',
    'driver_quality_3y', 'team_avg_pos_3y', 'team_trend'
]
target_col = 'pct_puntos_final'

# ==================== 2. CARGAR Y ENTRENAR ====================
print("Cargando dataset histórico completo (2008-2024)...")
df = pd.read_csv(INPUT_FILE)

# Asegurar orden
df = df.sort_values(['year', 'round', 'driver']).reset_index(drop=True)

X = df[feature_cols]
y = df[target_col]

print(f"Entrenando modelo maestro con {len(df)} registros...")
print(f"Configuración: {best_params}")

# Entrenar modelo final (sin splits, usamos todo)
final_model = xgb.XGBRegressor(**best_params)
final_model.fit(X, y)

print("✅ Modelo final entrenado.")

# ==================== 3. GUARDAR PARA LA APP ====================
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# 1. Guardar el modelo
with open(FINAL_MODEL_FILE, 'wb') as f:
    pickle.dump(final_model, f)

# 2. Guardar la lista de features (vital para que la App no falle)
with open(f'{OUTPUT_DIR}/final_features_list.pkl', 'wb') as f:
    pickle.dump(feature_cols, f)

print(f"\n💾 ARCHIVOS GENERADOS:")
print(f"   -> Modelo:   {FINAL_MODEL_FILE}")
print(f"   -> Features: {OUTPUT_DIR}/final_features_list.pkl")
print("\n¡LISTO! Ahora actualiza tu app_v6.py para cargar este modelo.")

Cargando dataset histórico completo (2008-2024)...
Entrenando modelo maestro con 7579 registros...
Configuración: {'subsample': 0.95, 'reg_alpha': 0.1, 'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.08, 'colsample_bytree': 0.5, 'objective': 'reg:squarederror', 'n_jobs': -1, 'random_state': 42}
✅ Modelo final entrenado.

💾 ARCHIVOS GENERADOS:
   -> Modelo:   ../models/xgboost_base_final.pkl
   -> Features: ../models/final_features_list.pkl

¡LISTO! Ahora actualiza tu app_v6.py para cargar este modelo.
